# Analysis

In [1]:
from file import read, write
import pandas as pd

In [41]:
games = read('data/games.json')

gamesToRemove = []
for i in range(len(games)):
    game = games[i]

    if 'start_time' not in game:
        gamesToRemove.append(i)

    remove = []
    for key, value in game.items():
        if type(value) == type("string"):
            remove.append(key)
    
    for key in remove:
        del game[key]

for index in sorted(gamesToRemove, reverse=True):
    del games[index]

games

[{'blue_adc_assists': 0,
  'blue_adc_damage_to_objectives': 1293.170654296875,
  'blue_adc_deaths': 4,
  'blue_adc_kills': 2,
  'blue_adc_minion_kills': 197,
  'blue_adc_vision_score': 13,
  'blue_deaths': 22,
  'blue_gold': 41352,
  'blue_jungle_assists': 3,
  'blue_jungle_damage_to_objectives': 1741.5931396484375,
  'blue_jungle_deaths': 1,
  'blue_jungle_kills': 1,
  'blue_jungle_minion_kills': 4,
  'blue_jungle_vision_score': 16,
  'blue_kills': 7,
  'blue_mid_assists': 1,
  'blue_mid_damage_to_objectives': 0,
  'blue_mid_deaths': 4,
  'blue_mid_kills': 3,
  'blue_mid_minion_kills': 227,
  'blue_mid_vision_score': 21,
  'blue_support_assists': 4,
  'blue_support_damage_to_objectives': 325.0124816894531,
  'blue_support_deaths': 7,
  'blue_support_kills': 0,
  'blue_support_minion_kills': 41,
  'blue_support_vision_score': 48,
  'blue_top_assists': 0,
  'blue_top_damage_to_objectives': 5834.8857421875,
  'blue_top_deaths': 6,
  'blue_top_kills': 1,
  'blue_top_minion_kills': 190,
  

In [61]:
df = pd.DataFrame(games)

# clean
df = df.dropna(axis='columns')

# normalize
for column in df.columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()     
    
df

75


,blue_adc_assists,blue_adc_damage_to_objectives,blue_adc_deaths,blue_adc_kills,blue_adc_minion_kills,blue_adc_vision_score,blue_deaths,blue_gold,blue_jungle_assists,blue_jungle_damage_to_objectives,...,red_support_vision_score,red_support_xp,red_top_assists,red_top_damage_to_objectives,red_top_deaths,red_top_kills,red_top_minion_kills,red_top_vision_score,red_top_xp,total
0,-1.431130,-1.191517,0.900031,-0.641908,-1.069656,-1.447294,1.101539,-1.341098,-0.850124,-1.816618,...,-0.579971,0.121126,1.316882,-0.664632,-1.437512,-0.298728,-1.159090,-0.600313,-0.734041,0.507415
1,-0.020458,1.221029,-0.240573,-0.006977,0.292817,-0.548710,-0.016657,-0.132915,0.536760,1.218470,...,-0.243518,-0.678514,-0.336439,-1.215363,1.168810,0.578093,-0.871951,-0.440317,-0.717917,0.507415
2,-1.148996,-0.605286,-0.240573,-0.324442,-0.682589,-1.559617,1.381088,-1.553444,-0.156682,-1.210626,...,-1.185587,-0.575388,0.765775,-0.021516,-1.437512,1.016504,-0.256654,-0.760309,-0.455842,0.507415
3,-0.866861,-0.926612,0.329729,-0.324442,-1.766374,-1.896585,0.821990,-1.856857,-0.156682,-0.586436,...,-0.950070,-0.748902,-0.060885,-0.274672,-0.394983,1.454915,-0.666852,-1.320296,-0.746646,0.507415
4,-1.148996,-1.233910,0.329729,-0.324442,-1.054173,-1.615778,1.101539,-1.302119,-0.387829,-1.574072,...,-1.051006,-0.312660,1.592435,-0.479130,-0.394983,1.454915,-0.707872,-1.480293,-0.447047,0.507415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23701,1.108079,0.761157,0.329729,0.310488,-1.781857,-0.885679,-0.855304,-0.874563,-0.618976,0.006851,...,-1.454750,-1.623841,-1.438653,-0.278679,-0.394983,0.139683,-0.974501,-1.480293,-1.521147,1.762375
23702,-0.584727,-0.775701,2.040635,-0.959373,-0.620659,-0.492549,1.800412,-1.145768,0.074465,-1.155516,...,-1.084651,-0.132189,1.041328,0.349806,-1.437512,2.331736,-1.056540,-0.040326,-0.347963,1.762375
23703,2.518750,0.182658,2.610937,1.897814,0.943088,1.248457,2.079961,1.231879,0.767907,0.010682,...,0.059290,1.486734,4.072417,0.609846,1.690074,1.893325,-0.830931,2.039625,0.600083,1.762375
23704,0.261676,-1.025200,2.610937,-0.959373,-0.620659,-0.548710,2.359510,-0.335538,-0.156682,-0.158919,...,-0.075291,0.409635,0.214668,0.117420,1.168810,2.770147,-0.625832,0.759655,0.137493,1.762375


In [51]:
import numpy as np
from sklearn.decomposition import PCA

In [53]:
pca = PCA(n_components=5, whiten=False)
pca.fit(df)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.22623843 0.20023497 0.0907221  0.03540099 0.02463077]
[634.21104962 596.65127362 401.61244311 250.87554519 209.26162893]
